# <a>Projeto 3 - Classificação com Scikit Learn e XGBoost</a>

Projeto desenvolvido durante o curso Jornada Cientista de Dados do Let's Data

# <a>Motivação</a>

Muitos problemas na carreira do Cientista de Dados são modelados como classificação. Quando a variável target é discreta, precisamos modelar como classificação. Podemos ter apenas duas classes (variável target binária) ou multiclasses.

Além de criar os modelos, treinar e validar nas bases adequadas e testar, vamos fazer também o tratamento de missing data, feature engineering e selecionar a melhor métrica para validação.

# <a>Objeto de Estudo</a>

Um problema recorrente em empresas é a retenção de clientes, como saber se o cliente vai ou não deixar a empresa é uma grande vantagem, pois podemos então criar uma estratégia de marketing, ou desconto em produtos/serviços para que o cliente continue. Esse tipo de problema é chamado de Previão de Churn (Churn Rate), e para resolvê-lo precisamos de uma base com histórico de clientes que deixaram a empresa, bem como suas características (features).

Para esse projeto usaremos uma base de dados ficticia criada pela equipe do Let's Data para a prática de classificação com Scikit Learn e XGBoost.

In [1]:
# Importando as bibliotecas necessárias:

import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# Configurando o pandas para mostrar todas as linhas e colunas:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Configurando o pandas para não mostrar notação científica:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
# Fazendo o imput da base de dados de uma companhia telefonica:

df_clientes = pd.read_csv('C:\\data_science\\projeto-classificacao\\data\\train.csv')
df_clientes.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.00,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.00,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.00,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.00,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [9]:
# Verificando quais são as nossas features:
df_clientes.columns

Index(['id', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [10]:
# Cada id deve corresponder a um cliente, vamos verificar:
df_clientes['id'].is_unique

True

# <a>Dicionário das Features:</a>

* ID: identificação do cliente
* Gender: gênero do cliente
* SeniorCitizen: indicação de idoso
* Partner: possui ou não parceiro
* Dependents: indicador de dependentes
* Tenure: quantos meses ele está na empresa
* PhoneService: se possui serviços de telefonia
* MultipleLines: possui mais de uma linha
* InternetService: se possui serviços de internet
* OnlineSecurity: se possui serviço de segurança online
* OnlineBackup: possui serviço de backup online
* DeviceProtection: possui serviço de proteção de equipamentos
* TechSupport: possui serviço de suporte técnico
* StreamingTV: possui serviço de streaming
* StreamingMovies: possui serviço de streaming
* Contract: tipo de contrato
* PaperlessBilling: cobrança via papel ou eletronica
* PaymentMethod: forma de pagamento
* MonthlyCharges: cobrança mensal
* TotalCharges: cobrança total
* Churn: indica se o cliente saiu ou não da empresa

In [11]:
# Transformando o id do cliente no índice:
df_clientes = df_clientes.set_index('id')
df_clientes.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
id,,,,,,,,,,,,,,,,,,,,
4030,Female,0,No,No,56.00,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
6479,Female,0,Yes,No,60.00,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
6861,Female,0,No,No,37.00,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
3266,Male,0,Yes,Yes,29.00,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [12]:
# Avaliando a quantidade de linhas e colunas:
df_clientes.shape

(5634, 20)

Temos então 19 features para nos ajudar a prever o churn (20ª coluna do DataFrame)

In [13]:
# Avaliando os tipos das features:
df_clientes.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [14]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5634 entries, 4030 to 103
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5634 non-null   object 
 1   SeniorCitizen     5634 non-null   int64  
 2   Partner           5634 non-null   object 
 3   Dependents        5416 non-null   object 
 4   tenure            5173 non-null   float64
 5   PhoneService      5634 non-null   object 
 6   MultipleLines     5634 non-null   object 
 7   InternetService   5634 non-null   object 
 8   OnlineSecurity    5634 non-null   object 
 9   OnlineBackup      5634 non-null   object 
 10  DeviceProtection  5634 non-null   object 
 11  TechSupport       5634 non-null   object 
 12  StreamingTV       5634 non-null   object 
 13  StreamingMovies   5634 non-null   object 
 14  Contract          5634 non-null   object 
 15  PaperlessBilling  5634 non-null   object 
 16  PaymentMethod     5535 non-null   object

Percebemos duas coisas importantes, a primeira é que existem dados faltantes em Dependents, tenure e PaymentMethod. A segunda, é que a feature TotalCharges deveria ser float, assim como a MonthlyCharges, pois ambas tratam do valor a ser cobrado.

In [15]:
df_clientes['TotalCharges'].dtype

dtype('O')

In [17]:
# Vamos tirar os espaços em branco primeiro, utilizando expressão regular simples \s:
df_clientes['TotalCharges'] = df_clientes['TotalCharges'].replace("\s+", "", regex=True)

In [18]:
# Tentando converter para float:
df_clientes['TotalCharges'].astype(float)

ValueError: could not convert string to float: ''

In [23]:
# Não funcionou provavelmente por conta de células vazias:
indice_total_charges_nulo = df_clientes.loc[df_clientes['TotalCharges'] == ''].index
indice_total_charges_nulo

Int64Index([6755, 5219, 6671, 937, 3827, 1083, 3332, 1341], dtype='int64', name='id')

In [26]:
# Temos 8 valores nulos nessa feature, fazendo a conversão e transformando para NaN:
pd.to_numeric(df_clientes['TotalCharges'], errors='coerce')[indice_total_charges_nulo]

id
6755   NaN
5219   NaN
6671   NaN
937    NaN
3827   NaN
1083   NaN
3332   NaN
1341   NaN
Name: TotalCharges, dtype: float64

In [28]:
# Convertendo:
df_clientes['TotalCharges'] = pd.to_numeric(df_clientes['TotalCharges'], errors='coerce')
df_clientes.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

Conversão feita com sucesso de objetc para float!